In [ ]:
# Diagnosis
# DDXPlus (10 types, each 40q; total 400q)
# top 1 reference + 1 llmself generated answer

In [ ]:
## Env setup (deepseek)
import getpass
import os
from langchain_community.graphs import Neo4jGraph
from langchain_deepseek import ChatDeepSeek
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from dotenv import load_dotenv

# LLM
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
#os.environ["OPENAI_BASE_URL"] = os.getenv('OPENAI_BASE_URL')
os.environ["DEEPSEEK_API_KEY"] = os.getenv('DEEPSEEK_API_KEY')
os.environ["DEEPSEEK_BASE_URL"] = os.getenv('DEEPSEEK_BASE_URL')

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# CSV env
import sys
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from dotenv import load_dotenv
import pandas as pd

sys.path.append(r'C:\Users\Sin Yee\Desktop\rag_techniques')

In [2]:
## Verifier & Summarizer Func (MCQ)
import json
import pandas as pd
import ast

## Verifier Agent
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate

verifier_prompt = ChatPromptTemplate.from_messages([ 
    ("system", 
        "You are a medical verifier AI.\n"
        "Your role is to evaluate whether each reference is **relevant to the symptoms** described in the original question.\n\n"
        "Instructions:\n"
        "1. Carefully read the symptom description in the question.\n"
        "2. For each agent's reference, check if it describes symptoms that match or strongly relate to the question.\n"
        "3. Do NOT exclude a reference just because it refers to a different disease — as long as the symptoms are relevant, it should be kept.\n"
        "4. Exclude any references that do not meaningfully align with the symptom description, even if they are medically valid.\n"
        "5. Focus only on symptom relevance — diagnosis accuracy will be handled separately.\n\n"
        "Output format:\n"
        "'selected_agents': [list of agent names with relevant references — do not modify the names]\n\n"
        "DO NOT respond with anything else — no reasoning, no explanations, no formatting changes."
    ),
    ("human", 
     "Original Question:\n{question}\n\n"
     "Retrieved Reference:\n{ref}")
])


verifier_chain = LLMChain(llm=llm, prompt=verifier_prompt)

def verifier_agent(question: str, agent_reference: str):
    verifier_result = verifier_chain.invoke({
        "question": question,
        "ref": agent_reference
    })
    print(f"[Verifier response]:{verifier_result['text']}")
    
    return verifier_result['text']

## Summarizer Agent (MCQ)
from typing import Dict, Any
from langchain.chat_models import ChatOpenAI

def summarizer_agent(verified_references: Dict[str, Dict[str, str]], query: str) -> Dict[str, Any]:
    # Step 1: Extract all reference text across all sub-questions and sources
    flattened_refs = []
    for subq, sources in verified_references.items():
        for source_name, content in sources.items():
            flattened_refs.append(f"[{source_name}] {content}")

    # Step 2: Combine all references into a block
    context_block = "\n\n".join(flattened_refs)

    options = "Anemia, Boerhaave, Cluster headache, GERD, Influenza, Myocarditis, Panic attack, Pericarditis, Pneumonia, Sarcoidosis"

    # Step 3: Prepare the prompt
    user_prompt = (
        f"You are a Medical QnA AI. Use the references as contexts to answer the question.\n\n"
        f"MUST Answer with your own knowledge, if the references are irrelevant or wrong.\n\n"
        f"Select the single most likely diagnosis from the options.\n\n"
        f"Respond ONLY with the disease name.\n\n"
        f"Do NOT include reference labels in your answer.\n\n"
        f"### References:\n{context_block}\n\n"
        f"### Question:\n{query}\n\n"
        f"### Options:\n{options}\n\n"
        f"### Final Answer:"
    )

    # Step 4: Compose messages and call LLM
    messages = [
        {"role": "system", "content": "You are a Medical QnA AI that selects the most accurate answer."},
        {"role": "user", "content": user_prompt}
    ]

    response = llm.invoke(messages)

    # Step 5: Return final result
    return {
        "final_answer": response.content.strip(),
        "references": verified_references
    }

C:\Users\Sin Yee\AppData\Local\Temp\ipykernel_22648\2468998297.py:30: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  verifier_chain = LLMChain(llm=llm, prompt=verifier_prompt)


In [ ]:
## Verifier-Summarizer (deepseek)
import json
df = pd.read_csv("../new_data/ddxplus_400_results.csv")
for i in range(len(df)):
    try:
        query = df.loc[i, "EVIDENCES"]
        if pd.notna(query):
            retrieved_references = ast.literal_eval(df.loc[i, "ds_reference"])
            if isinstance(retrieved_references, dict):
                # convert unicode to chi characters
                decoded_value = {
                    k: json.loads(f'"{v}"') if isinstance(v, str) else v
                    for k, v in retrieved_references.items()
                }
            retrieved_references = decoded_value
            print(retrieved_references)
            verified_references = {}
            
            for question, agent_references in retrieved_references.items():
                print("="*50)
                print(f"Processing question: {question}")
                
                # Run verifier agent for current question and its references
                verifier_results = verifier_agent(question, agent_references)
                
                # Extract selected references based on verifier decision for current question
                selected_references = {}
                verifier_content = verifier_results.lower()
                
                # Simple parsing - look for agent names mentioned in the verifier result
                for agent_name, reference in agent_references.items():
                    if agent_name.lower() in verifier_content:
                        selected_references[agent_name] = reference
                
                # Store selected references for this question
                verified_references[question] = selected_references
                
                print(f"Selected agents for this question: {list(selected_references.keys())}")
            
            verified_references_str = json.dumps(verified_references, ensure_ascii=False) # convert dict to string
            df.loc[i, "ds_ver_ref"] = verified_references_str
            print("[Verified references]: ", verified_references_str)
            result = summarizer_agent(verified_references, query)
            print("[Summarizer answer]: ", result["final_answer"])
            df.loc[i, "ds"] = result["final_answer"]
            df.to_csv("../new_data/ddxplus_400_results.csv", index=False, encoding="utf-8-sig")

    except Exception as e:
        print(f"Error at row {i}: {e}")
        continue

In [ ]:
## Env setup (gpt-4o-mini)
import getpass
import os
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from dotenv import load_dotenv

# LLM
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
#os.environ["OPENAI_BASE_URL"] = os.getenv('OPENAI_BASE_URL')

llm = ChatOpenAI(model="gpt-4o-mini")

# CSV env
import sys
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from dotenv import load_dotenv
import pandas as pd

sys.path.append(r'C:\Users\Sin Yee\Desktop\rag_techniques')

In [ ]:
## Verifier & Summarizer Func (MCQ)
import json
import pandas as pd
import ast

## Verifier Agent
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate

verifier_prompt = ChatPromptTemplate.from_messages([ 
    ("system", 
        "You are a medical verifier AI.\n"
        "Your role is to evaluate whether each reference is **relevant to the symptoms** described in the original question.\n\n"
        "Instructions:\n"
        "1. Carefully read the symptom description in the question.\n"
        "2. For each agent's reference, check if it describes symptoms that match or strongly relate to the question.\n"
        "3. Do NOT exclude a reference just because it refers to a different disease — as long as the symptoms are relevant, it should be kept.\n"
        "4. Exclude any references that do not meaningfully align with the symptom description, even if they are medically valid.\n"
        "5. Focus only on symptom relevance — diagnosis accuracy will be handled separately.\n\n"
        "Output format:\n"
        "'selected_agents': [list of agent names with relevant references — do not modify the names]\n\n"
        "DO NOT respond with anything else — no reasoning, no explanations, no formatting changes."
    ),
    ("human", 
     "Original Question:\n{question}\n\n"
     "Retrieved Reference:\n{ref}")
])


verifier_chain = LLMChain(llm=llm, prompt=verifier_prompt)

def verifier_agent(question: str, agent_reference: str):
    verifier_result = verifier_chain.invoke({
        "question": question,
        "ref": agent_reference
    })
    print(f"[Verifier response]:{verifier_result['text']}")
    
    return verifier_result['text']

## Summarizer Agent (MCQ)
from typing import Dict, Any
from langchain.chat_models import ChatOpenAI

def summarizer_agent(verified_references: Dict[str, Dict[str, str]], query: str) -> Dict[str, Any]:
    # Step 1: Extract all reference text across all sub-questions and sources
    flattened_refs = []
    for subq, sources in verified_references.items():
        for source_name, content in sources.items():
            flattened_refs.append(f"[{source_name}] {content}")

    # Step 2: Combine all references into a block
    context_block = "\n\n".join(flattened_refs)

    options = "Anemia, Boerhaave, Cluster headache, GERD, Influenza, Myocarditis, Panic attack, Pericarditis, Pneumonia, Sarcoidosis"

    # Step 3: Prepare the prompt
    user_prompt = (
        f"You are a Medical QnA AI. Use the references as contexts to answer the question.\n\n"
        f"Answer with your internal knowledge, if the references are irrelevant or wrong.\n\n"
        f"Select the single most likely diagnosis from the options.\n\n"
        f"Respond ONLY with the disease name.\n\n"
        f"Do NOT include reference labels in your answer.\n\n"
        f"### References:\n{context_block}\n\n"
        f"### Question:\n{query}\n\n"
        f"### Options:\n{options}\n\n"
        f"### Final Answer:"
    )

    # Step 4: Compose messages and call LLM
    messages = [
        {"role": "system", "content": "You are a Medical QnA AI that selects the most accurate answer."},
        {"role": "user", "content": user_prompt}
    ]

    response = llm.invoke(messages)

    # Step 5: Return final result
    return {
        "final_answer": response.content.strip(),
        "references": verified_references
    }

In [ ]:
## Verifier-Summarizer (gpt)
import json
df = pd.read_csv("../new_data/ddxplus_400_results.csv")
for i in range(len(df)):
    try:
        query = df.loc[i, "EVIDENCES"]
        if pd.notna(query):
            retrieved_references = ast.literal_eval(df.loc[i, "gpt_reference"])
            if isinstance(retrieved_references, dict):
                # convert unicode to chi characters
                decoded_value = {
                    k: json.loads(f'"{v}"') if isinstance(v, str) else v
                    for k, v in retrieved_references.items()
                }
            retrieved_references = decoded_value
            print(retrieved_references)
            verified_references = {}
            
            for question, agent_references in retrieved_references.items():
                print("="*50)
                print(f"Processing question: {question}")
                
                # Run verifier agent for current question and its references
                verifier_results = verifier_agent(question, agent_references)
                
                # Extract selected references based on verifier decision for current question
                selected_references = {}
                verifier_content = verifier_results.lower()
                
                # Simple parsing - look for agent names mentioned in the verifier result
                for agent_name, reference in agent_references.items():
                    if agent_name.lower() in verifier_content:
                        selected_references[agent_name] = reference
                
                # Store selected references for this question
                verified_references[question] = selected_references
                
                print(f"Selected agents for this question: {list(selected_references.keys())}")
            
            verified_references_str = json.dumps(verified_references, ensure_ascii=False) # convert dict to string
            df.loc[i, "gpt_ver_ref"] = verified_references_str
            print("[Verified references]: ", verified_references_str)
            result = summarizer_agent(verified_references, query)
            print("[Summarizer answer]: ", result["final_answer"])
            df.loc[i, "gpt"] = result["final_answer"]
            df.to_csv("../new_data/ddxplus_400_results.csv", index=False, encoding="utf-8-sig")

    except Exception as e:
        print(f"Error at row {i}: {e}")
        continue

In [ ]:
# Diagnosis
# SymptomsDisease (10 types; 486q)
# top 5 references
# https://huggingface.co/datasets/sajjadhadi/disease-diagnosis-dataset

In [ ]:
## Verifier & Summarizer Func (MCQ)
import json
import pandas as pd
import ast

## Verifier Agent
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate

verifier_prompt = ChatPromptTemplate.from_messages([ 
    ("system", 
        "You are a medical verifier AI.\n"
        "Your role is to evaluate whether each reference is **relevant to the symptoms** described in the original question.\n\n"
        "Instructions:\n"
        "1. Carefully read the symptom description in the question.\n"
        "2. For each agent's reference, check if it describes symptoms that match or strongly relate to the question.\n"
        "3. Do NOT exclude a reference just because it refers to a different disease — as long as the symptoms are relevant, it should be kept.\n"
        "4. Exclude any references that do not meaningfully align with the symptom description, even if they are medically valid.\n"
        "5. Focus only on symptom relevance — diagnosis accuracy will be handled separately.\n\n"
        "Output format:\n"
        "'selected_agents': [list of agent names with relevant references — do not modify the names]\n\n"
        "DO NOT respond with anything else — no reasoning, no explanations, no formatting changes."
    ),
    ("human", 
     "Original Question:\n{question}\n\n"
     "Retrieved Reference:\n{ref}")
])


verifier_chain = LLMChain(llm=llm, prompt=verifier_prompt)

def verifier_agent(question: str, agent_reference: str):
    verifier_result = verifier_chain.invoke({
        "question": question,
        "ref": agent_reference
    })
    print(f"[Verifier response]:{verifier_result['text']}")
    
    return verifier_result['text']

## Summarizer Agent (MCQ)
from typing import Dict, Any
from langchain.chat_models import ChatOpenAI

def summarizer_agent(verified_references: Dict[str, Dict[str, str]], query: str) -> Dict[str, Any]:
    # Step 1: Extract all reference text across all sub-questions and sources
    flattened_refs = []
    for subq, sources in verified_references.items():
        for source_name, content in sources.items():
            flattened_refs.append(f"[{source_name}] {content}")

    # Step 2: Combine all references into a block
    context_block = "\n\n".join(flattened_refs)

    options = "anemia, allergy, fibromyalgia, gastroesophageal reflux disease (gerd), gout, heart failure, rheumatoid arthritis, spondylosis, schizophrenia, thrombophlebitis"

    # Step 3: Prepare the prompt
    user_prompt = (
        f"You are a Medical QnA AI. Use the references as contexts to answer the question.\n\n"
        f"Select the single most likely diagnosis from the options.\n\n"
        f"Respond ONLY with the disease name.\n\n"
        f"Do NOT include reference labels in your answer.\n\n"
        f"### References:\n{context_block}\n\n"
        f"### Question:\n{query}\n\n"
        f"### Options:\n{options}\n\n"
        f"### Final Answer:"
    )

    # Step 4: Compose messages and call LLM
    messages = [
        {"role": "system", "content": "You are a Medical QnA AI that selects the most accurate answer."},
        {"role": "user", "content": user_prompt}
    ]

    response = llm.invoke(messages)

    # Step 5: Return final result
    return {
        "final_answer": response.content.strip(),
        "references": verified_references
    }

In [ ]:
## Verifier-Summarizer (deepseek)
import json
df = pd.read_csv("../new_data/sajjadhadi_486_result.csv")
for i in range(len(df)):
    try:
        query = df.loc[i, "text"]
        if pd.notna(query):
            retrieved_references = ast.literal_eval(df.loc[i, "reference"])
            if isinstance(retrieved_references, dict):
                # convert unicode to chi characters
                decoded_value = {
                    k: json.loads(f'"{v}"') if isinstance(v, str) else v
                    for k, v in retrieved_references.items()
                }
            retrieved_references = decoded_value
            print(retrieved_references)
            verified_references = {}
            
            for question, agent_references in retrieved_references.items():
                print("="*50)
                print(f"Processing question: {question}")
                
                # Run verifier agent for current question and its references
                verifier_results = verifier_agent(question, agent_references)
                
                # Extract selected references based on verifier decision for current question
                selected_references = {}
                verifier_content = verifier_results.lower()
                
                # Simple parsing - look for agent names mentioned in the verifier result
                for agent_name, reference in agent_references.items():
                    if agent_name.lower() in verifier_content:
                        selected_references[agent_name] = reference
                
                # Store selected references for this question
                verified_references[question] = selected_references
                
                print(f"Selected agents for this question: {list(selected_references.keys())}")
            
            verified_references_str = json.dumps(verified_references, ensure_ascii=False) # convert dict to string
            df.loc[i, "ds_ver_ref"] = verified_references_str
            print("[Verified references]: ", verified_references_str)
            result = summarizer_agent(verified_references, query)
            print("[Summarizer answer]: ", result["final_answer"])
            df.loc[i, "ds"] = result["final_answer"]
            df.to_csv("../new_data/sajjadhadi_486_result.csv", index=False, encoding="utf-8-sig")

    except Exception as e:
        print(f"Error at row {i}: {e}")
        continue

In [ ]:
## Verifier-Summarizer (gpt)
import json
df = pd.read_csv("../new_data/sajjadhadi_486_result.csv")
for i in range(len(df)):
    try:
        query = df.loc[i, "text"]
        if pd.notna(query):
            retrieved_references = ast.literal_eval(df.loc[i, "reference"])
            if isinstance(retrieved_references, dict):
                # convert unicode to chi characters
                decoded_value = {
                    k: json.loads(f'"{v}"') if isinstance(v, str) else v
                    for k, v in retrieved_references.items()
                }
            retrieved_references = decoded_value
            print(retrieved_references)
            verified_references = {}
            
            for question, agent_references in retrieved_references.items():
                print("="*50)
                print(f"Processing question: {question}")
                
                # Run verifier agent for current question and its references
                verifier_results = verifier_agent(question, agent_references)
                
                # Extract selected references based on verifier decision for current question
                selected_references = {}
                verifier_content = verifier_results.lower()
                
                # Simple parsing - look for agent names mentioned in the verifier result
                for agent_name, reference in agent_references.items():
                    if agent_name.lower() in verifier_content:
                        selected_references[agent_name] = reference
                
                # Store selected references for this question
                verified_references[question] = selected_references
                
                print(f"Selected agents for this question: {list(selected_references.keys())}")
            
            verified_references_str = json.dumps(verified_references, ensure_ascii=False) # convert dict to string
            df.loc[i, "gpt_ver_ref"] = verified_references_str
            print("[Verified references]: ", verified_references_str)
            result = summarizer_agent(verified_references, query)
            print("[Summarizer answer]: ", result["final_answer"])
            df.loc[i, "gpt"] = result["final_answer"]
            df.to_csv("../new_data/sajjadhadi_486_result.csv", index=False, encoding="utf-8-sig")

    except Exception as e:
        print(f"Error at row {i}: {e}")
        continue

In [ ]:
# Diagnosis
# Symptom2Disease (22 types, test = 212q)
# top 1 reference + 1 llmself generated answer
# https://huggingface.co/datasets/gretelai/symptom_to_diagnosis/

In [ ]:
## Verifier & Summarizer Func (MCQ)
import json
import pandas as pd
import ast

## Verifier Agent
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate

verifier_prompt = ChatPromptTemplate.from_messages([ 
    ("system", 
        "You are a medical verifier AI.\n"
        "Your role is to evaluate whether each reference is **relevant to the symptoms** described in the original question.\n\n"
        "Instructions:\n"
        "1. Carefully read the symptom description in the question.\n"
        "2. For each agent's reference, check if it describes symptoms that match or strongly relate to the question.\n"
        "3. Do NOT exclude a reference just because it refers to a different disease — as long as the symptoms are relevant, it should be kept.\n"
        "4. Exclude any references that do not meaningfully align with the symptom description, even if they are medically valid.\n"
        "5. Focus only on symptom relevance — diagnosis accuracy will be handled separately.\n\n"
        "Output format:\n"
        "'selected_agents': [list of agent names with relevant references — do not modify the names]\n\n"
        "DO NOT respond with anything else — no reasoning, no explanations, no formatting changes."
    ),
    ("human", 
     "Original Question:\n{question}\n\n"
     "Retrieved Reference:\n{ref}")
])


verifier_chain = LLMChain(llm=llm, prompt=verifier_prompt)

def verifier_agent(question: str, agent_reference: str):
    verifier_result = verifier_chain.invoke({
        "question": question,
        "ref": agent_reference
    })
    print(f"[Verifier response]:{verifier_result['text']}")
    
    return verifier_result['text']

## Summarizer Agent (MCQ)
from typing import Dict, Any
from langchain.chat_models import ChatOpenAI

def summarizer_agent(verified_references: Dict[str, Dict[str, str]], query: str) -> Dict[str, Any]:
    # Step 1: Extract all reference text across all sub-questions and sources
    flattened_refs = []
    for subq, sources in verified_references.items():
        for source_name, content in sources.items():
            flattened_refs.append(f"[{source_name}] {content}")

    # Step 2: Combine all references into a block
    context_block = "\n\n".join(flattened_refs)

    options = "drug reaction, allergy, chicken pox, diabetes, psoriasis, hypertension, cervical spondylosis, bronchial asthma, varicose veins, malaria, dengue, arthritis, impetigo, fungal infection, common cold, gastroesophageal reflux disease, urinary tract infection, typhoid, pneumonia, peptic ulcer disease, jaundice, migraine"

    # Step 3: Prepare the prompt
    user_prompt = (
        f"You are a Medical QnA AI. Use the references as contexts to answer the question.\n\n"
        f"Use ur internal knowledge if the contexts are irrelevant or wrong.\n\n"
        f"Select the single most likely diagnosis from the options.\n\n"
        f"Respond ONLY with the disease name.\n\n"
        f"Do NOT include reference labels in your answer.\n\n"
        f"### References:\n{context_block}\n\n"
        f"### Question:\n{query}\n\n"
        f"### Options:\n{options}\n\n"
        f"### Final Answer:"
    )

    # Step 4: Compose messages and call LLM
    messages = [
        {"role": "system", "content": "You are a Medical QnA AI that selects the most accurate answer."},
        {"role": "user", "content": user_prompt}
    ]

    response = llm.invoke(messages)

    # Step 5: Return final result
    return {
        "final_answer": response.content.strip(),
        "references": verified_references
    }

In [ ]:
## Verifier-Summarizer (DeepSeek)
import json
df = pd.read_csv("../new_data/symp_diag_212_result.csv")
for i in range(len(df)):
    try:
        query = df.loc[i, "input"]
        if pd.notna(query):
            retrieved_references = ast.literal_eval(df.loc[i, "ds_reference"])
            if isinstance(retrieved_references, dict):
                # convert unicode to chi characters
                decoded_value = {
                    k: json.loads(f'"{v}"') if isinstance(v, str) else v
                    for k, v in retrieved_references.items()
                }
            retrieved_references = decoded_value
            print(retrieved_references)
            verified_references = {}
            
            for question, agent_references in retrieved_references.items():
                print("="*50)
                print(f"Processing question: {question}")
                
                # Run verifier agent for current question and its references
                verifier_results = verifier_agent(question, agent_references)
                
                # Extract selected references based on verifier decision for current question
                selected_references = {}
                verifier_content = verifier_results.lower()
                
                # Simple parsing - look for agent names mentioned in the verifier result
                for agent_name, reference in agent_references.items():
                    if agent_name.lower() in verifier_content:
                        selected_references[agent_name] = reference
                
                # Store selected references for this question
                verified_references[question] = selected_references
                
                print(f"Selected agents for this question: {list(selected_references.keys())}")
            
            verified_references_str = json.dumps(verified_references) # convert dict to string
            df.loc[i, "ds_ver_ref"] = verified_references_str
            print("[Verified references]: ", verified_references_str)
            result = summarizer_agent(verified_references, query)
            print("[Summarizer answer]: ", result["final_answer"])
            df.loc[i, "ds"] = result["final_answer"]
            df.to_csv("../new_data/symp_diag_212_result.csv", index=False, encoding="utf-8-sig")

    except Exception as e:
        print(f"Error at row {i}: {e}")
        continue

In [ ]:
## Verifier-Summarizer (GPT)
import json
df = pd.read_csv("../new_data/symp_diag_212_result.csv")
for i in range(len(df)):
    try:
        query = df.loc[i, "input"]
        if pd.notna(query):
            retrieved_references = ast.literal_eval(df.loc[i, "gpt_reference"])
            if isinstance(retrieved_references, dict):
                # convert unicode to chi characters
                decoded_value = {
                    k: json.loads(f'"{v}"') if isinstance(v, str) else v
                    for k, v in retrieved_references.items()
                }
            retrieved_references = decoded_value
            print(retrieved_references)
            verified_references = {}
            
            for question, agent_references in retrieved_references.items():
                print("="*50)
                print(f"Processing question: {question}")
                
                # Run verifier agent for current question and its references
                verifier_results = verifier_agent(question, agent_references)
                
                # Extract selected references based on verifier decision for current question
                selected_references = {}
                verifier_content = verifier_results.lower()
                
                # Simple parsing - look for agent names mentioned in the verifier result
                for agent_name, reference in agent_references.items():
                    if agent_name.lower() in verifier_content:
                        selected_references[agent_name] = reference
                
                # Store selected references for this question
                verified_references[question] = selected_references
                
                print(f"Selected agents for this question: {list(selected_references.keys())}")
            
            verified_references_str = json.dumps(verified_references) # convert dict to string
            df.loc[i, "gpt_ver_ref"] = verified_references_str
            print("[Verified references]: ", verified_references_str)
            result = summarizer_agent(verified_references, query)
            print("[Summarizer answer]: ", result["final_answer"])
            df.loc[i, "gpt"] = result["final_answer"]
            df.to_csv("../new_data/symp_diag_212_result.csv", index=False, encoding="utf-8-sig")

    except Exception as e:
        print(f"Error at row {i}: {e}")
        continue